<a href="https://colab.research.google.com/github/daniellybx/bioest_aplicada_r/blob/main/modelo_holt_winters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de séries temporais Holt-Winters

In [ ]:
#IMPORTANDO PACOTES UTILIZADOS
import pickle
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from dateutil.parser import parse 
from pandas.plotting import autocorrelation_plot, lag_plot
from pandas.api.types import CategoricalDtype
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
#IMPORTANDO DADOS
serie = pd.read_csv(os.path.expanduser('incluir_csv_aqui.csv'), parse_dates =['variavel_data'])
serie.info(), serie.head()

In [ ]:
#PLOTANDO GRÁFICO COM VALORES POR DIA
tab = pd.DataFrame(serie.groupby('variavel_data')['variavel_frequencia'].sum())

sns.set(rc={'figure.figsize':(17, 10)})
sns.set_theme()
line = sns.lineplot(data = tab, x = tab.index, y = 'variavel_frequencia')
line.set(xlabel='Data', ylabel='Deals', title= "Número de deals por dia")

In [ ]:
#AGRUPANDO DADOS POR SEMANA E POR MÊS
tab2 = tab.copy()
tab2 = tab2.reset_index()
tab2['week_day'] = tab2['variavel_data'].dt.weekday 
tab2['week'] = tab2['variavel_data'].dt.strftime('%Y-w%U')
tab2['month'] = [d.strftime('%m') for d in tab2.variavel_data]

In [ ]:
#CRIANDO BOXPLOT DA DISTRIBUICAO DE DIÁRIA
sns.set(rc={'figure.figsize':(5, 4)})
sns.set_theme()
box = sns.boxplot(y="variavel_frequencia", data=tab2)
box.set(xlabel='', ylabel='Frequência', title= "Distribuição por dia")

In [ ]:
#REMOVENDO OUTLIERS SUPERIORES (>99%)
mean = tab2.loc[tab2['variavel_frequencia'] < tab2['variavel_frequencia'].quantile(0.99)].mean()
tab2.loc[tab2['variavel_frequencia'] > tab2['variavel_frequencia'].quantile(0.99)] = np.nan
tab2.fillna(mean, inplace=True)

tab3 = tab2.copy()
tab3['week'] = tab3['variavel_data'].dt.strftime('%U')
tab3 = tab3.set_index(['variavel_data'])

#CRIANDO BOXPLOT DA DISTRIBUICAO POR DIA
sns.set(rc={'figure.figsize':(5, 4)})
sns.set_theme()
box = sns.boxplot(y="variavel_frequencia", data=tab3)
box.set(xlabel='', ylabel='Frequência', title= "Distribuição por dia")

In [ ]:
#DESENHANDO UM GRÁFICO DE LINHAS POR SEMANA (INCLUIR MÊS SE FOR O CASO)
weeks = tab3['week'].unique()
np.random.seed(100)
mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(weeks), replace=False)

dict_week = {0:'Segunda', 1:'Terça', 2:'Quarta', 3:'Quinta', 4:'Sexta', 5:'Sábado', 6:'Domingo'} 
tab3['weeks'] = tab3['week_day'].replace(dict_week)

plt.figure(figsize=(17, 10), dpi= 80)

for i, y in enumerate(weeks):
    if i > 0:        
        plt.plot('weeks', 'variavel_frequencia', data=tab3.loc[tab3.week==y, :], color=mycolors[i], label=y)

# Decoration
plt.gca().set(xlim=(0, 6), ylim=(0, 90), ylabel='$variavel_frequencia$', xlabel='Dia da semana')
plt.yticks(fontsize=12, alpha=.7)
plt.title("Tendências semanais", fontsize=20)
plt.show()

In [ ]:
#DESENHANDO UM GRÁFICO DE LINHAS POR MÊS
months = tab2['month'].unique()
np.random.seed(100)
mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(months), replace=False)

plt.figure(figsize=(17, 10), dpi= 80)

for i, y in enumerate(weeks):
    if i > 0:        
        plt.plot('month', 'variavel_frequencia', data=tab2.loc[tab2.month==y, :], color=mycolors[i], label=y)

# Decoration
plt.gca().set(xlim=(0, 6), ylim=(0, 90), ylabel='$variavel_frequencia$', xlabel='Mês')
plt.yticks(fontsize=12, alpha=.7)
plt.title("Tendências mensais", fontsize=20)
plt.show()

In [ ]:
#BOXPLOT COM DISPERSÃO POR SEMANA E MÊS
fig, axes = plt.subplots(1, 2, figsize=(20,7), dpi= 80)

box_week = sns.boxplot(x='week', y='variavel_frequencia', data=tab3, ax=axes[0])
box_day = sns.boxplot(x='weeks', y='variavel_frequencia', data=tab3, order = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado'], ax=axes[1])

box_week.set(ylabel='variavel_frequencia', xlabel='Semana do ano', title= "Distribuição semana do ano") 
box_day.set(ylabel='variavel_frequencia', xlabel='Dia da semana', title= "Distribuição dia da semana") 
plt.show()

In [ ]:
#DECOMPONDO OS COMPONENTES DA SÉRIE
# tab_mul = seasonal_decompose(tab['total_deals'], model='multiplicative', extrapolate_trend='freq')
tab_add = seasonal_decompose(tab['total_deals'], model='additive', extrapolate_trend='freq')

### Treinando um modelo de séries temporais Holt-Winters

In [ ]:
#PLOTANDO OS GRÁFICOS DA DECOMPOSIÇÃO
#tab_mul.plot().suptitle('Multiplicativo')
tab_add.plot().suptitle('Aditivo')
plt.show()

In [ ]:
#EXTRAINDO OS COMPONETES DA SÉRIE 
## ESCOLHA DE SÉRIE ADITIVA COM BASE NOS RESÍDUOS MAIS PRÓXIMOS DE 0
tab_ts_add = pd.concat([round(tab_add.seasonal,1), round(tab_add.trend,1), round(tab_add.resid,1), round(tab_add.observed,1)], axis=1)
tab_ts_add.columns = ['sazonalidade', 'tendencia', 'residuos', 'total_deals']
tab_ts_add.head()

In [ ]:
#TESTANDO ESTACIONARIEDADE DA SÉRIE 

# ADF Test
## H0 = ST possui raiz unitária e é não estacionária
result = adfuller(tab.total_deals, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

In [ ]:
#TESTANDO A SAZONALIDADE 
plt.rcParams.update({'figure.figsize':(7,4), 'figure.dpi':120})
autocorrelation_plot(tab.total_deals.tolist())

In [ ]:
#AUTOCORRELAÇÃO E AUTOCORRELAÇÃO PARCIAL
fig, axes = plt.subplots(1,2, figsize=(17,5), dpi= 100)
plot_acf(tab.total_deals.tolist(), lags=50, ax=axes[0])
plot_pacf(tab.total_deals.tolist(), lags=50, ax=axes[1])
plt.show()

In [ ]:
#LAG PLOTS
plt.rcParams.update({'ytick.left' : False, 'axes.titlepad':10})

# Plot
fig, axes = plt.subplots(1, 4, figsize=(10,3), sharex=True, sharey=True, dpi=100)

for i, ax in enumerate(axes.flatten()[:4]):
    lag_plot(tab.total_deals, lag=i+1, ax=ax, c='firebrick')
    ax.set_title('Lag ' + str(i+1))

fig.suptitle('Lag Plots para o total de deals iClinic', y=1.15)    

### Predição com modelo Holt-Winters

In [ ]:
#IMPORTANDO PACOTES
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
#DEFININDO TREINO E TESTE
tab_modelo = pd.DataFrame(tab['total_deals'].copy())
train = tab_modelo[:150] #INCLUIR NÚMERO DE OBSERVAÇOES ATÉ 70%
test = tab_modelo[150:] #INCLUIR NÚMERO DE OBSERVAÇOES MAIORES DE 70%

In [ ]:
#ESCOLHENDO MODELO FINAL 
fitted_model = ExponentialSmoothing(train['total_deals'], 
                                    seasonal= 'add', 
                                    trend = 'add', 
                                    seasonal_periods=7
                                    ).fit()

simulations = fitted_model.simulate(28, repetitions=1000, error="add")

plt.figure(figsize=(17, 10), dpi= 80)

test_predictions = fitted_model.forecast(28)
train['variavel_frequencia'].plot(legend=True,label='Treinamento')
test['variavel_frequencia'].plot(legend=True,label='Teste',figsize=(6,4))
test_predictions.plot(legend=True,label='Predição')
plt.title('Treino, teste e predição modelo Holt Winters')

In [ ]:
ax = train.plot(
    figsize=(10, 6),
    marker="o",
    color="black",
    title="Predição e simulações para modelo aditivo Holt-Winters",
)

ax.set_ylabel("Nome da Frequencia")
ax.set_xlabel("Mês")
fitted_model.fittedvalues.plot(ax=ax, style="--", color="green")
simulations.plot(ax=ax, style="-", alpha=0.05, color="grey", legend=False)
fitted_model.forecast(28).rename("Holt-Winters").plot(
    ax=ax, style="--", marker="o", color="green", legend=True
)
plt.show()

In [ ]:
#INTERVALO DE CONFIANCA
upper_ci = simulations.quantile(q=0.95, axis='columns')
lower_ci = simulations.quantile(q=0.05, axis='columns')

In [ ]:
#RESULTADOS DO MODELO
print(fitted_model.summary())

In [ ]:
#RESULTADOS
test['valor_estimado'] = round(test_predictions)
test['menor_valor_esperado_95'] = round(lower_ci)
test['menor_valor_esperado_95'] = np.where((test['menor_valor_esperado_95'] < 0), 0, test['menor_valor_esperado_95'])
test['maior_valor_esperado_95'] = round(upper_ci)
test['true_value'] = np.where((test['variavel_frequencia'] >= test['menor_valor_esperado_95']) & (test['variavel_frequencia'] <= test['maior_valor_esperado_95']), 1, 0)
test['dia_semana'] = test.index
test['dia_semana'] = test['dia_semana'].dt.weekday 
test['dia_semana'] = test['dia_semana'].replace(dict_week)

In [ ]:
#ERROS QUADRADOS DA PREDIÇÃO 
deals = test['variavel_frequencia']
pred = test['valor_estimado']
print(f'Mean Absolute Error = {mean_absolute_error(deals, pred)}')
print(f'Mean Squared Error = {mean_squared_error(deals, pred)}')

In [ ]:
#MÉDIA ESPERADA POR DIA DA SEMANA
days = CategoricalDtype(categories= ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado'], ordered=True)
test['dia_semana'] = test['dia_semana'].astype(days)
mean_day_week = test.groupby('dia_semana')[['valor_estimado', 'menor_valor_esperado_95', 'maior_valor_esperado_95']].agg(['mean','std']).round(1)
mean_day_week

In [ ]:
#SALVANDO O MODELO EM PKG
with open("modelo_final.pkg", 'wb') as file:  
    pickle.dump(fitted_model, file)